In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
import os
from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper
from Utils.transform import ToCategorical, cutOut

Num GPUs Available: 1


Using TensorFlow backend.


In [2]:
dimension = (64,64)
batch_size = 300
channels = 5
optimizer = Adam( lr = 1e-3 )
slices = [256,320,256,320]
cutOutFrame = cutOut(slices)

categorical_list = [0,1,5,10,15,30,60,120]
categorical = ToCategorical(categorical_list)

PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATIONS = [categorical]
TRANSFORMATIONS = None

def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)


def provideData(flatten=False,dimension=dimension,batch_size=60,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          transform=TRANSFORMATIONS,
                          preTransformation=PRETRAINING_TRANSFORMATIONS)

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [10]:
def testzero(input_shape,
           n_predictions=1,
           simpleclassification=None,
           flatten_output=False,
           activation_hidden="relu",
           activation_output="relu"):

    def zeroInflatedPoisson(output):
        rate = tf.math.exp(output[:,:,:,0:1]) #A 
        s = tf.math.sigmoid(output[:,:,:,1:2])
        
        pos = tfp.distributions.Poisson(rate=rate)
        det = tfp.distributions.Deterministic(loc=tf.zeros_like(rate))

        components = [det,pos]
        mixture = tfd.Mixture(
              cat=tfd.Categorical(probs=tf.stack([1-s, s],axis=-1)),#D
              components=components)
        return tfp.distributions.Independent(mixture,reinterpreted_batch_ndims=1,name="ZeroInflated")
    
    inputs = Input(shape=input_shape) 

    conv01 = Conv2D(10, kernel_size=(3, 3), padding="same")(inputs)       # 10 x 64x64
    conv01 = Activation(activation_hidden)(conv01)
    conv01_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv01)            # 10 x 32x32


    conv02 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv01_pool)  # 20 x 32x32
    conv02 = Activation(activation_hidden)(conv02)
    conv02_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv02)            # 20 x 16x16


    conv03 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv02_pool)  # 20 x 16x16
    conv03 = Activation(activation_hidden)(conv03)
    conv03_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv03)            # 20 x 8x8


    conv04 = Conv2D(20, kernel_size=(3, 3), padding="same")(conv03_pool)  # 20 x 8x8
    conv04 = Activation(activation_hidden)(conv04)
    conv04_pool = MaxPooling2D((2, 2), strides=(2, 2))(conv04)            # 20 x 4x4


    ### UPSAMPLING:
    up04 = UpSampling2D((2, 2))(conv04_pool)    # 20 x 8x8
    up04 = concatenate([conv04, up04], axis=3)  # 20+20 x 8x8


    up03 = UpSampling2D((2, 2))(up04)           # 40 x 16x16
    up03 = concatenate([conv03, up03], axis=3)  # 20+40 x 16x16


    up02 = UpSampling2D((2, 2))(up03)           # 60 x 32x32
    up02 = concatenate([conv02, up02], axis=3)  # 20+60 x 32x32


    up01 = UpSampling2D((2, 2))(up02)           # 80 x 64x64
    up01 = concatenate([conv01, up01], axis=3)  # 10+80 x 64x64


    output = Conv2D(2, (1, 1))(up01)  # 1 x 64x64

    output = tfp.layers.DistributionLambda(zeroInflatedPoisson,name="ZPoisson")(output)
    model = Model(inputs=inputs, outputs=output)
    return model

In [11]:
t = Trainer(testzero,
                    NLL,
                    (train,test),
                    batch_size = batch_size,
                    optimizer=optimizer,
                    dimension = dimension,
                    channels = channels,
                    metrics = ["mse","mae"])

print("len train,val",len(train),len(test))

[Load file failed]  ./model_data/testzero_function/testzero_function64x64x5.h5
[Load file failed]  ./model_data/testzero_function/testzero_function64x64x5history.json
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 64, 64, 5)]  0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 64, 64, 10)   460         input_4[0][0]                    
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 64, 64, 10)   0           conv2d_15[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPoo

In [12]:
t.fit(200)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 31 steps, validate for 31 steps
Epoch 1/200
 1/31 [..............................] - ETA: 5:59

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[63213,31,64,64,1] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator gpu_host_bfc
	 [[node model_2/ZPoisson/model_2_ZPoisson_ZeroInflated/value/model_2_ZPoisson_ZeroInflated/sample/model_2_ZPoisson_Mixture/sample/model_2_ZPoisson_Poisson/sample/random_poisson/RandomPoissonV2 (defined at /home_ext/ios/anaconda3/envs/DeepRain/lib/python3.6/site-packages/tensorflow_probability/python/distributions/poisson.py:200) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[63213,31,64,64,1] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator gpu_host_bfc
	 [[node model_2/ZPoisson/model_2_ZPoisson_ZeroInflated/value/model_2_ZPoisson_ZeroInflated/sample/model_2_ZPoisson_Mixt